SHOE STORE
Finding the best location

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

!conda install -c conda-forge folium=0.5.0 --yes # the Foursquare API lab
import folium # map rendering library

!pip install xlrd
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [2]:
df=pd.read_excel('met_10r_3gdp.xls', index_col=0, skiprows=8) #Gross domestic product (GDP) at current market prices by
                                                              #metropolitan regions.

Let's have a look at our dataframe

In [3]:
df.head(20)

,2013,2014,2015,2016,2017
METROREG/TIME,,,,,
Belgium,392340,400087,411010,424660,439052
Bruxelles / Brussel,119418,120651,124921,129224,132124
Antwerpen,44330.1,45601.8,46960.3,48411.1,50172.5
Gent,24428,25314.8,26364.9,27429.7,28434.4
Charleroi,10985.1,11127.2,11200,11324.5,11754.6
Liège,18717,18977,19201.1,19791.6,20541.8
Non-metropolitan regions in Belgium,174191,178146,182104,188298,195831
Bulgaria,41858.1,42824.4,45288.5,48128.6,51663
Sofia,18164.5,18518.8,19956.1,21266.1,22852.6


Let's clear our data a little bit

In [4]:
  # reset the index, the old index is added as a column, and a new sequential index is used
df.reset_index(inplace=True) 

  # user friendly column name
df.rename(columns={"METROREG/TIME": "City"}, inplace=True) 

  # Remove rows with missing value in City column 
df.dropna(subset=["City"], axis=0, inplace=True)

  # Eurostat uses ":" for years without obseravtions, let replace ":" to NaN
df.replace(":", np.nan, inplace = True)

  # reset the index, this time no need to add a new column
df.reset_index(drop=True, inplace=True)

In [5]:
#let's see what we get
df.tail(20)

,City,2013,2014,2015,2016,2017
328,Blackburn - Blackpool - Preston,18605.2,19978.38,24713.92,23398.17,22597.88
329,Middlesbrough,12777,13887.42,15643.40,13958.70,13437.44
330,Oxford,25998.2,28992.48,33938.79,30955.76,29824.72
331,Norwich,9965.02,10807.20,12191.79,11370.61,10869.18
332,Cheshire West and Chester,19367.1,21130.68,24531.76,23009.47,22831.70
333,Ipswich,20044.7,21968.85,24859.02,22768.59,22252.27
334,Non-metropolitan regions in United Kingdom,447524,492396.90,563494.18,515114.13,498476.55
335,Norway,393966,376647.60,348408.40,335747.50,NaN
336,Oslo,97961,96240.08,93234.41,95106.68,NaN
337,Bergen,64904,63324.93,61488.46,58239.24,NaN


In [6]:
df1 = df.drop([df.index[346], df.index[347]]) #drop the last two rows, because they contain system information

As we can see in the table with cities GDP Eurostat included data for countries and Non-metropolitan regions, which we need to clear. Let's create a list with European countries.

In [7]:
countries = ['Austria', 'Bulgaria', 'Italy', 'Belgium', 'Latvia', 'Germany',
            'Lithuania', 'Croatia', 'Luxembourg', 'Cyprus', 'Malta', 'Czech Republic',
            'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Denmark',
            'Czechia', 'Netherlands', 'Spain', 'Estonia', 'Greece', 'Montenegro',
            'Finland', 'Norway', 'Albania', 'Turkey', 'Serbia', 'North Macedonia',
            'Hungary', 'Sweden', 'Ireland', 'United Kingdom', 'France']

Now, we'll create a list with indices of rows containing country name in City column

In [8]:
unwanted_indexes=[] # create a list
for i in range(len(df1)): 
    for country in countries:
        if country in (df1.loc[i,'City']): #checking if there is a city name in City or country and region
            unwanted_indexes.append(i) #creating our list         

In [9]:
print(unwanted_indexes) #let's check what we've got

[0, 6, 7, 12, 13, 18, 19, 24, 25, 94, 95, 97, 98, 101, 102, 105, 106, 130, 131, 154, 166, 167, 170, 171, 193, 194, 196, 198, 199, 202, 203, 204, 205, 211, 212, 214, 215, 229, 230, 236, 237, 257, 258, 262, 263, 273, 274, 277, 278, 281, 282, 286, 287, 292, 293, 334, 335, 338, 339, 340, 341, 342, 343, 344, 345]


In [10]:
df1.drop(df1.index[unwanted_indexes], inplace=True) #remove rows with data not related to cities

Let's check whether we have missing values in our table.

In [11]:
missing_data = df1.isnull()
missing_data.tail(5)

,City,2013,2014,2015,2016,2017
331,False,False,False,False,False,False
332,False,False,False,False,False,False
333,False,False,False,False,False,False
336,False,False,False,False,False,True
337,False,False,False,False,False,True


In [12]:
for column in missing_data.columns.values.tolist(): #count the rows with missing data
    print(column)
    print (missing_data[column].value_counts())
    print("")  

City
False    281
Name: City, dtype: int64

2013
False    216
True      65
Name: 2013, dtype: int64

2014
False    235
True      46
Name: 2014, dtype: int64

2015
False    280
True       1
Name: 2015, dtype: int64

2016
False    280
True       1
Name: 2016, dtype: int64

2017
True     216
False     65
Name: 2017, dtype: int64



There are no GDP data for 1 city in 2016 and 65 cities in 2017, let replace NaN with value 
from previous year.

In [13]:
df1["2016"].replace(np.nan, df1['2015'], inplace=True)

In [14]:
df1['2017'].replace(np.nan, df1['2016'], inplace=True)

In [15]:
df1

,City,2013,2014,2015,2016,2017
1,Bruxelles / Brussel,119418,120650.69,124921.15,129224.22,132123.86
2,Antwerpen,44330.1,45601.85,46960.27,48411.06,50172.51
3,Gent,24428,25314.76,26364.92,27429.66,28434.40
4,Charleroi,10985.1,11127.21,11200.02,11324.51,11754.60
5,Liège,18717,18976.99,19201.12,19791.64,20541.76
...,...,...,...,...,...,...
331,Norwich,9965.02,10807.20,12191.79,11370.61,10869.18
332,Cheshire West and Chester,19367.1,21130.68,24531.76,23009.47,22831.70
333,Ipswich,20044.7,21968.85,24859.02,22768.59,22252.27
336,Oslo,97961,96240.08,93234.41,95106.68,95106.68


Let's find the top cities by GDP. We will use data for 2017, because it's the newest.

In [16]:
df1.sort_values(by=['2017'], ascending=False, inplace=True)

In [17]:
city_gdp=df1.iloc[[0,1,2,3,4], [0, 5]]  #create dataframe with 5 top European cities by GDP for 2017
city_gdp.reset_index(drop=True, inplace=True)
city_gdp

,City,2017
0,London,763494.39
1,Paris,683962.00
2,Madrid,211782.00
3,Milano,200116.42
4,München,178945.00


In [18]:
city_gdp.rename(columns={"2017": "GDP"}, inplace=True) #further we'll use data for 2017
city_gdp

,City,GDP
0,London,763494.39
1,Paris,683962.00
2,Madrid,211782.00
3,Milano,200116.42
4,München,178945.00


In [19]:
top_cities=city_gdp['City'].tolist() #let's make a list with our top 5 cities
top_cities

['London', 'Paris', 'Madrid', 'Milano', 'München']

Now we have a dataframe with top 5 cities in Europe by GDP. 

The target group for "TP Collection" is aged 25-35, so let's try to find out the GDP for for this age group in each city.
From Eurostat we can get the data for total population of top cities and population aged 25-35 for the same cities. 
Using simple arithmetic calculations we can get the GDP for population aged 25-35.

In [20]:
df_yp= pd.read_excel('urb_cpop1.xls', index_col=0, skiprows=8)  #data for young population aged 25-35 for Europe
df_ttl= pd.read_excel('urb_cpop1 total.xls', index_col=0, skiprows=8) #data for total population for Europe
#for both files we have to skip the first 8 rows with system info

In [21]:
  # reset the index, the old index is added as a column, and a new sequential index is used
df_yp.reset_index(inplace=True) 
df_ttl.reset_index(inplace=True) 

  # user friendly column name
df_yp.rename(columns={"CITIES/TIME": "City"}, inplace=True)
df_ttl.rename(columns={"CITIES/TIME": "City"}, inplace=True)

  # Remove rows with missing value in City column 
df_yp.dropna(subset=["City"], axis=0, inplace=True)
df_ttl.dropna(subset=["City"], axis=0, inplace=True)

  # Eurostat uses ":" for years without obseravtions, let replace ":" to NaN
df_yp.replace(":", np.nan, inplace = True)
df_ttl.replace(":", np.nan, inplace = True)

  # reset the index, this time no need to add a new column
df_yp.reset_index(drop=True, inplace=True)
df_ttl.reset_index(drop=True, inplace=True)

In [22]:
df_yp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078 entries, 0 to 1077
Data columns (total 5 columns):
City    1077 non-null object
2016    826 non-null object
2017    754 non-null float64
2018    731 non-null float64
2019    37 non-null float64
dtypes: float64(3), object(2)
memory usage: 42.2+ KB


In [23]:
df_ttl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078 entries, 0 to 1077
Data columns (total 4 columns):
City    1077 non-null object
2016    844 non-null object
2017    754 non-null float64
2018    731 non-null float64
dtypes: float64(2), object(2)
memory usage: 33.8+ KB


Both dataframes consists of 1078 entries, so we can process them simultaneously.

In [24]:
yp_ind=[] # create a list for top cities indices in young population
ttl_ind=[] # create a list for top cities indices in total population
for i in range(len(df_yp)-2): 
    for t_city in top_cities:
        if t_city in (df_yp.loc[i,'City']): #checking if this is our top city in City
            yp_ind.append(i) #creating our list with top city indexes
        if t_city in (df_ttl.loc[i,'City']): #checking if this is our top city in City
            ttl_ind.append(i) #creating our list with top city indexes    

In [25]:
# drop all rows that are not related to our 5 top cities
df_yp = df_yp.iloc[yp_ind]
df_ttl = df_ttl.iloc[ttl_ind]

In [26]:
# reset the index, no need to add a new column
df_yp.reset_index(drop=True, inplace=True)
df_ttl.reset_index(drop=True, inplace=True)

In [27]:
df_yp

,City,2016,2017,2018,2019
0,München,271272,273702.0,267864.0,NaN
1,Madrid,418732,414425.0,420788.0,NaN
2,Madrid,645076,630076.0,628193.0,NaN
3,"Rozas de Madrid, Las",10573,10272.0,9958.0,NaN
4,Paris,1497168,NaN,NaN,NaN
5,Paris (greater city),NaN,NaN,NaN,NaN
6,CA le Parisis,NaN,NaN,NaN,NaN
7,Milano,160802,161447.0,163680.0,NaN
8,Milano (greater city),447832,444657.0,443827.0,NaN
9,London (greater city),1691837,1669610.0,1644020.0,NaN


In [28]:
df_ttl

,City,2016,2017,2018
0,München,1450381,1464301.0,1456039.0
1,Madrid,3165541,3182981.0,3223334.0
2,Madrid,4878277,4904291.0,4955432.0
3,"Rozas de Madrid, Las",94471,95071.0,95550.0
4,Paris,9803494,NaN,NaN
5,Paris (greater city),NaN,NaN,NaN
6,CA le Parisis,NaN,NaN,NaN
7,Milano,1345851,1351562.0,1366180.0
8,Milano (greater city),4074585,4087060.0,4106356.0
9,London (greater city),8730803,8797330.0,8866541.0


The dataframe is still not perfect: we have several entries for the same city and total of 12 rows of data for 5 cities! 
Eurostat include data for cities and greater cities too and there are cases when cities names include other city names...
People from greater cities could be clients for "TP collections" too, so let everywhere use data for greater cities and drop unnecessary data. 

In [29]:
df_yp.drop(df_yp.index[[1,3,5,6,7,10]], inplace=True)
df_ttl.drop(df_ttl.index[[1,3,5,6,7,10]], inplace=True)
df_yp.reset_index(drop=True, inplace=True)
df_ttl.reset_index(drop=True, inplace=True)

In [30]:
df_yp

,City,2016,2017,2018,2019
0,München,271272,273702.0,267864.0,NaN
1,Madrid,645076,630076.0,628193.0,NaN
2,Paris,1497168,NaN,NaN,NaN
3,Milano (greater city),447832,444657.0,443827.0,NaN
4,London (greater city),1691837,1669610.0,1644020.0,NaN


In [31]:
df_ttl

,City,2016,2017,2018
0,München,1450381,1464301.0,1456039.0
1,Madrid,4878277,4904291.0,4955432.0
2,Paris,9803494,NaN,NaN
3,Milano (greater city),4074585,4087060.0,4106356.0
4,London (greater city),8730803,8797330.0,8866541.0


In [32]:
# For Paris we don't have data for 2017 and 2018, so let's copy it from 2016 in both datarframes
df_ttl.iloc[2,3]=df_ttl.iloc[2,1]
df_yp.iloc[2,3]=df_yp.iloc[2,1]

In [33]:
df_yp.drop(columns=['2016', '2017', '2019'], inplace=True) #drop unnecessary data for 2016, 2017 and column 
                                                            #for 2019 - there are only NaN.
df_ttl.drop(columns=['2016', '2017'], inplace=True)
df_yp.sort_values(by=['2018'], ascending=False, inplace=True)
df_ttl.sort_values(by=['2018'], ascending=False, inplace=True)
df_yp.reset_index(drop=True, inplace=True)
df_ttl.reset_index(drop=True, inplace=True)

In [34]:
df_yp.rename(columns={"2018": "POP_25-35"}, inplace=True) #further we'll use data for 2018

In [35]:
df_ttl.loc[df_ttl.City =='Milano (greater city)', 'City'] = 'Milano'
df_ttl.loc[df_ttl.City =='London (greater city)', 'City'] = 'London'
df_ttl.rename(columns={"2018": "POP_total"}, inplace=True) #further we'll use data for 2018
df_ttl

,City,POP_total
0,Paris,9803494.0
1,London,8866541.0
2,Madrid,4955432.0
3,Milano,4106356.0
4,München,1456039.0


And the last thing, let's rename Milano (greater city) to Milano, the same procedure with London

In [37]:
df_yp.loc[df_yp.City =='Milano (greater city)', 'City'] = 'Milano'
df_yp.loc[df_yp.City =='London (greater city)', 'City'] = 'London'

In [38]:
# In order to calculate the proportion of population 25-34 to the total population for each of top 5 cities
# we have to divide values from datafrom df_yp to df_ttl, but we can't do it directly because cities are not in the same order
# let's arrange dataframes once more time
df_yp.sort_values(by=['City'], ascending=False, inplace=True)
df_ttl.sort_values(by=['City'], ascending=False, inplace=True)
df_yp.loc[:,'POP_25-35'] = df_yp.loc[:,'POP_25-35']/df_ttl.loc[:,'POP_total'] 

In [39]:
df_yp

,City,POP_25-35
1,Paris,0.168856
4,München,0.183968
3,Milano,0.108083
2,Madrid,0.126769
0,London,0.167697


OK, now we have the top 5 cities in Europe by GDP in city_gdp and proportion of population 25-34 to total population df_yp in these cities.
This data will help us to understand the purchasing power of population 25-34 in each city.

In [40]:
city_gdp

,City,GDP
0,London,763494.39
1,Paris,683962.00
2,Madrid,211782.00
3,Milano,200116.42
4,München,178945.00


In [42]:
gdp_final = pd.merge(df_yp, city_gdp, how='left', on=['City']) # merge two dataframes
gdp_final['gdp_25_35']=gdp_final['GDP']*gdp_final['POP_25-35']        # calculate the GDP for population aged 25-35
gdp_final.sort_values(by=['gdp_25_35'], ascending=False, inplace=True)
gdp_final.reset_index(drop=True, inplace=True)
gdp_final

,City,POP_25-35,GDP,gdp_25_35
0,London,0.167697,763494.39,128035.988704
1,Paris,0.168856,683962.00,115491.037555
2,München,0.183968,178945.00,32920.082141
3,Madrid,0.126769,211782.00,26847.300079
4,Milano,0.108083,200116.42,21629.169594


Now we have a list with 5 top European cities based on purchasing power of population aged 25-35.
As we can see, London is at the top with Paris very close to it.
The next city Munich, has only about one fourth of London's purchasing power for this group!

Our next metric is purchasing power of tourists, visiting our top European cities. 
From Eurostat web page we can get information about number of nights spent in the city by tourists per year, which is very close to our metric.  
Let's arrange top cities according number of nights spent in the city by tourists per year.

In [43]:
df_tour = pd.read_excel('urb_ctour.xls', index_col=0, skiprows=8) #Excel file with number of nights spent by torists in Europe

In [44]:
df_tour.reset_index(inplace=True)
df_tour.tail(15)

,CITIES/TIME,2014,2015,2016
924,Utrecht,:,:,:
925,Eindhoven,:,:,:
926,Arnhem,:,:,:
927,Heerlen,:,:,:
928,Haarlem,:,:,:
929,Leiden,:,:,:
930,Ede,:,:,:
931,Alkmaar,:,:,:
932,Belfast,:,:,:
933,Derry,:,:,:


In [45]:
df_tour.rename(columns={"CITIES/TIME": "City"}, inplace=True)

In [46]:
df_tour.head()

,City,2014,2015,2016
0,Belgium,:,38380415,34083462
1,Bruxelles / Brussel,:,6443213,5210495
2,Antwerpen,:,:,:
3,Gent,:,:,:
4,Charleroi,:,242703,247045


Let's filter the data and select entries related to our top cities

In [47]:
top_cities=['London', 'Paris', 'Madrid', 'Milano', 'München']
tour_ind=[] # create a list for top countries indices
for i in range(len(df_tour)-3): 
    for t_city in top_cities:
            if t_city in (df_tour.loc[i,'City']): #checking if this is our top city in City
                tour_ind.append(i) #creating our list with top city indexes 

In [48]:
df_tour = df_tour.iloc[tour_ind] #dropping unecessary cities

In [49]:
df_tour.reset_index(drop=True, inplace=True)
  # Eurostat uses ":" for years without obseravtions, let replace ":" to NaN
df_tour.replace(":", np.nan, inplace = True)
df_tour

,City,2014,2015,2016
0,München,13448024.0,14066212.0,14041090.0
1,Madrid,17723714.0,19292547.0,19844222.0
2,"Rozas de Madrid, Las",NaN,NaN,NaN
3,Paris,14960029.0,14126682.0,13058510.0
4,CA le Parisis,120732.0,117918.0,107777.0
5,Milano,10232396.0,11741374.0,10976244.0
6,City of London,NaN,NaN,NaN


Again resulting dataframe is not perfect - we've got data for cities containing names from our list (we already know how to handle them), but more surprisingly there is no data for London....

In [50]:
df_tour.drop(df_tour.index[[2,4]], inplace=True) #drop rows for other cities
df_tour.reset_index(drop=True, inplace=True)
df_tour.drop(columns=['2014', '2015'], inplace=True) #drop columns for previous years
df_tour.loc[df_tour.City =='City of London', 'City'] = 'London' #rename London
df_tour.rename(columns={"2016": "Tourists"}, inplace=True) #rename column with nights spent simply to Tourists 
df_tour

,City,Tourists
0,München,14041090.0
1,Madrid,19844222.0
2,Paris,13058510.0
3,Milano,10976244.0
4,London,NaN


In [51]:
# Let's replace NaN for London with mean for other 4 cities
df_tour["Tourists"].replace(np.nan, df_tour['Tourists'].astype('float').mean(axis=0), inplace=True)
# normalize data
df_tour['Tourists'] = df_tour['Tourists']/df_tour['Tourists'].max() #normalize data
df_tour.sort_values(by=['Tourists'], ascending=False, inplace=True)
df_tour

,City,Tourists
1,Madrid,1.000000
4,London,0.729684
0,München,0.707566
2,Paris,0.658051
3,Milano,0.553120


OK, at this point we have the top 5 cities in Europe by GDP in df_gdp, proportion of population 25-34 to total population df_yp in these cities.
This data will help us to understand the purchasing power of population 25-34 in each city. Additionally we've got data for tourists.
These data will help us to estimate the purchasing potential of cities.  


The next criteria is so called 'index of doing business' An economy’s ease of doing business score is reflected on a scale from 0 to 100, where 0 represents the lowest and 100 represents the best performance.

In [52]:
df_dtf1 = pd.read_excel('Rankings.xlsx', index_col=0) #Excel file with data from the World bank

In [53]:
df_dtf1.head()

,globalRank,Rank within group,Starting a Business,Dealing with Construction Permits,Getting Electricity,Registering Property,Getting Credit,Protecting Minority Investors,Paying Taxes,Trading across Borders,Enforcing Contracts,Resolving Insolvency
Economy,,,,,,,,,,,,
New Zealand,1.0,1.0,1.0,7.0,48.0,2.0,1.0,3.0,9.0,63.0,23.0,36.0
Singapore,2.0,2.0,4.0,5.0,19.0,21.0,37.0,3.0,7.0,47.0,1.0,27.0
"Hong Kong SAR, China",3.0,3.0,5.0,1.0,3.0,51.0,37.0,7.0,2.0,29.0,31.0,45.0
Denmark,4.0,4.0,45.0,4.0,21.0,11.0,48.0,28.0,8.0,1.0,14.0,6.0
"Korea, Rep.",5.0,5.0,33.0,12.0,2.0,40.0,67.0,25.0,21.0,36.0,2.0,11.0


In [54]:
df_dtf=df_dtf1[['Starting a Business']] # for "TP collection" overall metric is no so important as starting a business
df_dtf.head()

,Starting a Business
Economy,
New Zealand,1.0
Singapore,4.0
"Hong Kong SAR, China",5.0
Denmark,45.0
"Korea, Rep.",33.0


In [55]:
df_dtf.reset_index(inplace=True)
df_dtf.head()

,Economy,Starting a Business
0,New Zealand,1.0
1,Singapore,4.0
2,"Hong Kong SAR, China",5.0
3,Denmark,45.0
4,"Korea, Rep.",33.0


As we can see the data is related to countries not to cities, so we have to create a list with countries where our top 5 cities belongs to.

In [56]:
top_cities

['London', 'Paris', 'Madrid', 'Milano', 'München']

In [57]:
top_countries=['United Kingdom', 'France', 'Spain', 'Italy', 'Germany']

In [58]:
tc_ind=[] # create a list for top countries indices
for i in range(len(df_dtf)): 
    for t_country in top_countries:
            if t_country in (df_dtf.loc[i,'Economy']): #checking if this is our top country
                tc_ind.append(i) #creating our list with top counties indexes 

In [59]:
tc_ind

[7, 21, 29, 31, 57]

In [60]:
# drop all rows that are not related to our 5 top cities
df_dtf = df_dtf.iloc[tc_ind]

In [61]:
df_dtf

,Economy,Starting a Business
7,United Kingdom,18.0
21,Germany,125.0
29,Spain,97.0
31,France,37.0
57,Italy,98.0


In [62]:
df_dtf.reset_index(drop=True, inplace=True)
    #replace the name of the country with respective city
df_dtf.loc[df_dtf.Economy =='France', 'Economy'] = 'Paris'
df_dtf.loc[df_dtf.Economy =='Germany', 'Economy'] = 'München'
df_dtf.loc[df_dtf.Economy =='Italy', 'Economy'] = 'Milano'
df_dtf.loc[df_dtf.Economy =='Spain', 'Economy'] = 'Madrid'
df_dtf.loc[df_dtf.Economy =='United Kingdom', 'Economy'] = 'London'
df_dtf.rename(columns={"Economy": "City"}, inplace=True)

In [63]:
df_dtf.sort_values(by=['Starting a Business'], inplace=True)  
df_dtf.reset_index(drop=True, inplace=True)
df_dtf

,City,Starting a Business
0,London,18.0
1,Paris,37.0
2,Madrid,97.0
3,Milano,98.0
4,München,125.0


The value in column "Starting a Business" is a position of the city in the global ranking, i.e. 1 is the top position. For our further analysis is good to have normalized data with values between 0 and 1 where the top is 1.

In [64]:
df_dtf['Starting a Business'] = 1 - df_dtf['Starting a Business']/df_dtf['Starting a Business'].max() #normalize data
df_dtf

,City,Starting a Business
0,London,0.856
1,Paris,0.704
2,Madrid,0.224
3,Milano,0.216
4,München,0.000


Now we have top cities ranked by ease of doingbusiness.

Let's look again at our previous two city rankings: by purchasing power of population 25-35 and purchasing power of tourists

In [66]:
gdp_final

,City,POP_25-35,GDP,gdp_25_35
0,London,0.167697,763494.39,128035.988704
1,Paris,0.168856,683962.00,115491.037555
2,München,0.183968,178945.00,32920.082141
3,Madrid,0.126769,211782.00,26847.300079
4,Milano,0.108083,200116.42,21629.169594


In [67]:
gdp_final.drop(columns=['POP_25-35', 'GDP'], inplace=True) # drop unnecessary columns
gdp_final['gdp_25_35'] = gdp_final['gdp_25_35']/gdp_final['gdp_25_35'].max() #normalize data
gdp_final

,City,gdp_25_35
0,London,1.000000
1,Paris,0.902020
2,München,0.257116
3,Madrid,0.209686
4,Milano,0.168930


In [68]:
df_tour

,City,Tourists
1,Madrid,1.000000
4,London,0.729684
0,München,0.707566
2,Paris,0.658051
3,Milano,0.553120


In [69]:
    # merge data for GDP 
city_rank=df_dtf.merge(gdp_final, left_on='City', right_on='City')
city_rank

,City,Starting a Business,gdp_25_35
0,London,0.856,1.000000
1,Paris,0.704,0.902020
2,Madrid,0.224,0.209686
3,Milano,0.216,0.168930
4,München,0.000,0.257116


In [70]:
    # merge data for population 25-35 
city_rank=city_rank.merge(df_tour, left_on='City', right_on='City')
city_rank

,City,Starting a Business,gdp_25_35,Tourists
0,London,0.856,1.000000,0.729684
1,Paris,0.704,0.902020,0.658051
2,Madrid,0.224,0.209686,1.000000
3,Milano,0.216,0.168930,0.553120
4,München,0.000,0.257116,0.707566


Now we have all required data in one dataframe and easily can calculate the overall ranking. 
It seems reasonable to set weight coefficients of 0.7 for metric purchasing power of population 25-35 and 0.15 for both metrics tourists ranking and Doing Business and find which city will get maximum score.

In [71]:
city_rank['Rank'] = city_rank['gdp_25_35']*0.7 + (city_rank['Starting a Business'] + city_rank['Tourists'])*0.15
city_rank['Rank'] = city_rank['Rank']/city_rank['Rank'].max() #normalize data
city_rank

,City,Starting a Business,gdp_25_35,Tourists,Rank
0,London,0.856,1.000000,0.729684,1.000000
1,Paris,0.704,0.902020,0.658051,0.891101
2,Madrid,0.224,0.209686,1.000000,0.352273
3,Milano,0.216,0.168930,0.553120,0.249100
4,München,0.000,0.257116,0.707566,0.305076


The overall ranking leader is London.

Now, let's find the best location in London.

In [72]:
# library for BeautifulSoup, for web scrapping
!pip install lxml
!pip install bs4
from bs4 import BeautifulSoup# library to handle data in a vectorized manner
import numpy as np# library for data analsysis
from pandas import DataFrame
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)# library to handle JSON files
import json
print('numpy, pandas, ..., imported...')
!pip -q install geopy
print('geopy installed...')# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
print('Nominatim imported...')# library to handle requests
import requests
print('requests imported...')# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize
print('json_normalize imported...')# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('matplotlib imported...')# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Kmeans imported...')# install the Geocoder
!pip -q install geocoder
import geocoder# import time
import time
!pip -q install folium
print('folium installed...')
import folium # map rendering library
print('folium imported...')
print('...Done')

     |████████████████████████████████| 5.8MB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 112kB 6.4MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
numpy, pandas, ..., imported...
geopy installed...
Nominatim imported...
requests imported...
json_normalize imported...
matplotlib imported...
Kmeans imported...
folium installed...
folium imported...
...Done


In [73]:
# We can import List of areas of London from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
areas_London_url = requests.get(url)
areas_London_url

<Response [200]>

In [74]:
# The response code is 200 which is OK and we can go further
london_areas_data = pd.read_html(areas_London_url.text) #extract data from web page
len(london_areas_data), type(london_areas_data)

(5, list)

In [75]:
london_areas_data

[                                                   0
 0  Map all coordinates in "Category:Areas of Lond...
 1                 Download coordinates as: KML · GPX,
                                            Location  \
 0                                        Abbey Wood   
 1                                             Acton   
 2                                         Addington   
 3                                        Addiscombe   
 4                                       Albany Park   
 5                                  Aldborough Hatch   
 6                                           Aldgate   
 7                                           Aldwych   
 8                                          Alperton   
 9                                           Anerley   
 10                                            Angel   
 11                                        Aperfield   
 12                                          Archway   
 13                                   Ardleigh Green 

In [76]:
# There are a lot of different information at the web page. The table, which we are looking for is in the section Nr.1
df_lon1=london_areas_data[1] #create data frame from the table 

In [77]:
df_lon1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 6 columns):
Location             533 non-null object
London borough       533 non-null object
Post town            533 non-null object
Postcode district    533 non-null object
Dial code            533 non-null object
OS grid ref          531 non-null object
dtypes: object(6)
memory usage: 25.1+ KB


In [78]:
df_lon1.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


First, let make some cleaning.
There are some useless columns like Dial code and OS grid ref. We can drop them safely.
But when we tried to drop them, we received warning "['Dial code'] not found in axis". So, there are no such column..... Maybe there are some invisible special characters in the column name? Let's get the names.

In [79]:
columnsNamesArr = df_lon1.columns.values
columnsNamesArr

array(['Location', 'London\xa0borough', 'Post town',
       'Postcode\xa0district', 'Dial\xa0code', 'OS grid ref'],
      dtype=object)

Our suggestion was correct... We have to state the column names exactly as they are.

In [80]:
df_lon2=df_lon1.drop(columns=['Dial\xa0code', 'OS grid ref'])                 #
df_lon2.rename(columns={'London\xa0borough': 'Borough'}, inplace=True)        # now everything is OK with column names
df_lon2.rename(columns={'Postcode\xa0district': 'Postcode'}, inplace=True)    #
df_lon2.head()

,Location,Borough,Post town,Postcode
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Addington,Croydon[8],CROYDON,CR0
3,Addiscombe,Croydon[8],CROYDON,CR0
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [81]:
# In Borough column some of entries have reference numbers in square brackets. Let's clear them.
df_lon2['Borough'] = df_lon2['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df_lon2.head()

,Location,Borough,Post town,Postcode
0,Abbey Wood,"Bexley, Greenwich",LONDON,SE2
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
2,Addington,Croydon,CROYDON,CR0
3,Addiscombe,Croydon,CROYDON,CR0
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


Now we can find location coordinates by Postcode using Geocoder

In [82]:
# Defining a function to use --> get_latlng()'''
def get_latlng(arcgis_geocoder):
    
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [84]:
postal_codes = df_lon2['Postcode']    
coordinates = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]
# It's a long process and may takes few minutes

In [85]:
coordinates

[[51.492450000000076, 0.12127000000003818],
 [51.51324000000005, -0.2674599999999714],
 [51.38475500000004, -0.051498623999975734],
 [51.38475500000004, -0.051498623999975734],
 [51.452067760578885, 0.1722299153689439],
 [51.50642000000005, -0.1272099999999341],
 [51.51200000000006, -0.08057999999994081],
 [51.51651000000004, -0.11967999999995982],
 [51.50642000000005, -0.1272099999999341],
 [51.41009000000008, -0.05682999999993399],
 [51.523610000000076, -0.09876999999994496],
 [51.27172802817503, 0.07471050704375404],
 [51.56393000000003, -0.12944999999996298],
 [51.50642000000005, -0.1272099999999341],
 [51.615680000000054, -0.2451099999999542],
 [51.63429000000008, -0.13365999999996347],
 [51.44822000000005, -0.1483899999999494],
 [51.499600000000044, -0.09612999999995964],
 [51.523610000000076, -0.09876999999994496],
 [51.53312000000005, 0.08407653200004006],
 [51.50642000000005, -0.1272099999999341],
 [51.47052000000008, 0.14670637700004363],
 [51.47457000000003, -0.2421199999999

In [86]:
df = DataFrame (coordinates,columns=['lat','lng']) #convert list with coordinates to dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 2 columns):
lat    533 non-null float64
lng    533 non-null float64
dtypes: float64(2)
memory usage: 8.5 KB


In [87]:
df_lon3=df_lon2.join(df)   #merge our table with corresponding coordinates

In [88]:
df_lon3.tail()

,Location,Borough,Post town,Postcode,lat,lng
528,Woolwich,Greenwich,LONDON,SE18,51.48207,0.07143
529,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,51.50642,-0.12721
530,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,51.50645,-0.23691
531,Yeading,Hillingdon,HAYES,UB4,51.50642,-0.12721
532,Yiewsley,Hillingdon,WEST DRAYTON,UB7,51.50642,-0.12721


Now we have a dataframe with coordinates of locations postcodes. 
If we look at columns with coordinates, especially the last rows, we can see, that some locations with different postcodes have identical coordinates. 
We are going to make further analysis based on coordinates of postcodes, so we can optimize our work by deleting duplicates coordinates (the pair 'lat' and 'lng') and avoid double work.

In [89]:
df_lon3 = df_lon3.drop_duplicates(['lat', 'lng'],  keep='last') #drop all duplicates except last occurence

In [90]:
df_lon3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 1 to 532
Data columns (total 6 columns):
Location     142 non-null object
Borough      142 non-null object
Post town    142 non-null object
Postcode     142 non-null object
lat          142 non-null float64
lng          142 non-null float64
dtypes: float64(2), object(4)
memory usage: 7.8+ KB


Stop! This is strange - we had 533 location and after dropping the duplicate coordinates there are only 142 left. Probably, there is something wrong - we "lost" the major part of the data and it's not very believable that so many of locations has two postcodes. Let's get back to the dataframe in its state before cleaning the duplicates, make a dataframe with duplicate locations coordinates again and try to find what's wrong.

In [91]:
df_lon3=df_lon2.join(df)  #create full dataframe again
df_lon3.tail(10) #look at the tail again

,Location,Borough,Post town,Postcode,lat,lng
523,Woodford,Redbridge,LONDON,"IG8, E18",51.589770,0.030520
524,Woodford Green,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8,51.608869,0.015320
525,Woodlands,Hounslow,ISLEWORTH,TW7,51.506420,-0.127210
526,Woodside,Croydon,CROYDON,CR0,51.384755,-0.051499
527,Woodside Park,Barnet,LONDON,N12,51.615920,-0.176740
528,Woolwich,Greenwich,LONDON,SE18,51.482070,0.071430
529,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,51.506420,-0.127210
530,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,51.506450,-0.236910
531,Yeading,Hillingdon,HAYES,UB4,51.506420,-0.127210
532,Yiewsley,Hillingdon,WEST DRAYTON,UB7,51.506420,-0.127210


As we can see, there are a lot of locations with completely different post codes, for example: TW7, KT4, UB4 and UB7 (that means they are located in different parts of London) and it's not possible all of them to have the same coordinates 51.506420, -0,127210. Obviously, due to some reason, for all postcodes outside of central part of London, arcgis_geocoder returns the same coordinates. 

In [92]:
# Select all duplicate rows based on lat and lng columns
duplicateRowsDF = df_lon3[df_lon3.duplicated(['lat', 'lng'])]
 
print("Duplicate Rows based on lat and lng are:", duplicateRowsDF, sep='\n')  

Duplicate Rows based on lat and lng are:
                                           Location  \
3                                        Addiscombe   
8                                          Alperton   
13                                   Ardleigh Green   
18                                         Barbican   
20                                      Barkingside   
23                                      Barnes Cray   
24                                      Barnet Gate   
29                                        Beckenham   
31                                        Becontree   
32                                  Becontree Heath   
33                                       Beddington   
37                                          Belmont   
38                                          Belmont   
41                                       Bermondsey   
42                                       Berrylands   
44         Bexley (also Old Bexley, Bexley Village)   
45               Bexleyh

As we can see, arcgis_geocoder returns identical coordinates for locations placed at the very different parts of London. If we continue our work based on this data, it will reduce the quality of our project. Definitely, we should not use it. Let see what other options we have.

One can find a lot of geocoder services on Internet, but most of them are paid, so let's try to find some other way to get the coordinates of London postcodes.
If we look more thoroughly at the same Wiki page https://en.wikipedia.org/wiki/List_of_areas_of_London, we can see the last table column 'OS grid ref' which stands for Ordnance Survey (National Grid reference system - a system of geographic grid references used in Great Britain), actually already contains hypelinks with location coordinates and they are in our usual format longitude and latitude.
Let's try to extract them.

In [93]:
# There is a library for web scrapping called BeautifulSoup
column_names = ['LatLng'] 
from bs4 import BeautifulSoup
import urllib.request
ll_coord=pd.DataFrame(columns=column_names)

resp = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_areas_of_London")  
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))

for link in soup.find_all('a', href=True):
    if 'geohack/en' in link['href']:  # all our links contain string 'geohack/en' - this helps us to avoid other hyperlinks
        ll_coord=ll_coord.append({'LatLng': link['href']}, ignore_index=True)
        

In [94]:
ll_coord.head()

,LatLng
0,https://tools.wmflabs.org/geohack/en/51.486480...
1,https://tools.wmflabs.org/geohack/en/51.510587...
2,https://tools.wmflabs.org/geohack/en/51.362931...
3,https://tools.wmflabs.org/geohack/en/51.381621...
4,https://tools.wmflabs.org/geohack/en/51.434925...


In [95]:
# We'got the whole hyperlink which contains longitude and latitude in it. Let's extract them.
ll_coord['LatLng'] = ll_coord['LatLng'].astype(str) #convert to string

# strip text before the coordinates
ll_coord['LatLng'] = ll_coord['LatLng'].str.strip('https://tools.wmflabs.org/geohack/en') 

# strip text after the coordinates
ll_coord['LatLng'] = ll_coord['LatLng'].str.rstrip('_region:GB_scale:25000?pagename=List_of_areas_of_London') 

In [96]:
ll_coord.head()

,LatLng
0,51.48648031512;0.10859224316653
1,51.51058775877;-0.26498902668911
2,51.362931128458;-0.02637373877941
3,51.381621885559;-0.068682165650808
4,51.434925966837;0.12492137518833


In [97]:
# let's split the column to latitude and latitude
ll_coord['Lat'] = ll_coord.LatLng.str.split('\;').str[0]
ll_coord['Lng'] = ll_coord.LatLng.str.split('\;').str[1]
ll_coord.head()

,LatLng,Lat,Lng
0,51.48648031512;0.10859224316653,51.48648031512,0.10859224316653
1,51.51058775877;-0.26498902668911,51.51058775877,-0.26498902668911
2,51.362931128458;-0.02637373877941,51.362931128458,-0.02637373877941
3,51.381621885559;-0.068682165650808,51.381621885559,-0.068682165650808
4,51.434925966837;0.12492137518833,51.434925966837,0.12492137518833


In [98]:
ll_coord.info()  # check the dimensions

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 3 columns):
LatLng    531 non-null object
Lat       531 non-null object
Lng       531 non-null object
dtypes: object(3)
memory usage: 12.6+ KB


In [99]:
df_lon3=df_lon2   #get back our dataframe from original
df_lon3.info()    # check the dimensions

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 4 columns):
Location     533 non-null object
Borough      533 non-null object
Post town    533 non-null object
Postcode     533 non-null object
dtypes: object(4)
memory usage: 16.8+ KB


It appears that we have different number of entries in our two dataframes - 533 in df_lon3 with locations and 531 in ll_coord. This requires our reaction.
One possible reason is, if there are empty rows at the column with coordinates 'OS Grid ref'. Let's check it.

In [100]:
# We will process again our dataframe the same way. This time we'l not drop 'OS Grid ref' - the column with coordinates
df_lon2=df_lon1.drop(columns=['Dial\xa0code'])                                #
df_lon2.rename(columns={'London\xa0borough': 'Borough'}, inplace=True)        # now it is OK with column name
df_lon2.rename(columns={'Postcode\xa0district': 'Postcode'}, inplace=True)    #
df_lon2.head()

,Location,Borough,Post town,Postcode,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",TQ205805
2,Addington,Croydon[8],CROYDON,CR0,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",TQ478728


In [101]:
missing_data = df_lon2.isnull() # False - there is a value, True - there is no value
missing_data.head()

,Location,Borough,Post town,Postcode,OS grid ref
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False


In [102]:
# Let's check every column for missing values
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

Location
False    533
Name: Location, dtype: int64

Borough
False    533
Name: Borough, dtype: int64

Post town
False    533
Name: Post town, dtype: int64

Postcode
False    533
Name: Postcode, dtype: int64

OS grid ref
False    531
True       2
Name: OS grid ref, dtype: int64



Now, the situation is clear. All columns have 533 values except 'OS grid ref' column, which has missing data in two rows. We need to drop these two rows, then the relations between locations dataframe and coordinates dataframe will be correct.

In [103]:
# simply drop whole row with NaN in "OS grid ref" column
df_lon2.dropna(subset=["OS grid ref"], axis=0, inplace=True)

# reset index, because we droped two rows
df_lon2.reset_index(drop=True, inplace=True)

In [104]:
df_lon2.info() # one more check

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 5 columns):
Location       531 non-null object
Borough        531 non-null object
Post town      531 non-null object
Postcode       531 non-null object
OS grid ref    531 non-null object
dtypes: object(5)
memory usage: 20.9+ KB


In [105]:
df_lon2.head()

,Location,Borough,Post town,Postcode,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",TQ205805
2,Addington,Croydon[8],CROYDON,CR0,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",TQ478728


In [106]:
df_lon2['Lat']=ll_coord['Lat']  # now, when the dataframes have equal entries, we can correctly append Lat and Lng columns.
df_lon2['Lng']=ll_coord['Lng']
df_lon2.head()

,Location,Borough,Post town,Postcode,OS grid ref,Lat,Lng
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,TQ465785,51.48648031512,0.10859224316653
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",TQ205805,51.51058775877,-0.26498902668911
2,Addington,Croydon[8],CROYDON,CR0,TQ375645,51.362931128458,-0.02637373877941
3,Addiscombe,Croydon[8],CROYDON,CR0,TQ345665,51.381621885559,-0.068682165650808
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",TQ478728,51.434925966837,0.12492137518833


In [107]:
# Clear reference numbers in square brackets in Borough column
df_lon2['Borough'] = df_lon2['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))

# now we can safely drop few more columns
df_lon3=df_lon2.drop(columns=['Post town', 'Postcode', 'OS grid ref']) 

df_lon3.head()

,Location,Borough,Lat,Lng
0,Abbey Wood,"Bexley, Greenwich",51.48648031512,0.10859224316653
1,Acton,"Ealing, Hammersmith and Fulham",51.51058775877,-0.26498902668911
2,Addington,Croydon,51.362931128458,-0.02637373877941
3,Addiscombe,Croydon,51.381621885559,-0.068682165650808
4,Albany Park,Bexley,51.434925966837,0.12492137518833


Now, when we have in dataframe coordinates for all postcode locations it's time to estimate attractiveness of each location for new "TP Collections" new shop.
Just to remind - we will make a list of neighborhoods ordered by number of shoe shops located there and another 5 lists of neighborhoods ordered by number of venues preferred by young professional: trending café, fashion restaurants, art-galleries, university campuses and tourist attractions.
We will use Foursquare API for that purpose.

In [108]:
CLIENT_ID = 'R2KGTVRZKZ3VDRZALP1NALDNEC0OB2ORFFTYGSQNS42AVRE2' # my Foursquare ID
CLIENT_SECRET = 'FUVPPKJRWMGNAX3NXCKPVI1ULOPVRCPRMYOKFP0BNXWXVVAD' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 75
Radius=500
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: R2KGTVRZKZ3VDRZALP1NALDNEC0OB2ORFFTYGSQNS42AVRE2
CLIENT_SECRET:FUVPPKJRWMGNAX3NXCKPVI1ULOPVRCPRMYOKFP0BNXWXVVAD


We create a function for identifying the venues and counting the total number of venues for that category to respective columns in our current dataframe. Here is the function:

In [109]:
def nr_venues(ID, category):
    for i in range(len(df_lon3)):
        p_lat =df_lon3.iloc[i,2]
        p_lon =df_lon3.iloc[i,3]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,p_lat,p_lon,VERSION,ID,Radius,LIMIT) 
        res = requests.get(url).json()
        venue = res['response']['venues']
        df_ven=json_normalize(venue)
        count = np.shape(df_ven)
        df_lon3.loc[i, category] = count[0]

In [110]:
# here are the new columns
df_lon3['Shoe store'] = np.nan
df_lon3['Art gallery'] = np.nan
df_lon3['College and University'] = np.nan
df_lon3['Cafe'] = np.nan
df_lon3['Vegetarian Vegan Restaurant'] = np.nan
df_lon3['Travel and Transport'] = np.nan
df_lon3.head()

,Location,Borough,Lat,Lng,Shoe store,Art gallery,College and University,Cafe,Vegetarian Vegan Restaurant,Travel and Transport
0,Abbey Wood,"Bexley, Greenwich",51.48648031512,0.10859224316653,NaN,NaN,NaN,NaN,NaN,NaN
1,Acton,"Ealing, Hammersmith and Fulham",51.51058775877,-0.26498902668911,NaN,NaN,NaN,NaN,NaN,NaN
2,Addington,Croydon,51.362931128458,-0.02637373877941,NaN,NaN,NaN,NaN,NaN,NaN
3,Addiscombe,Croydon,51.381621885559,-0.068682165650808,NaN,NaN,NaN,NaN,NaN,NaN
4,Albany Park,Bexley,51.434925966837,0.12492137518833,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
df_lon3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 10 columns):
Location                       531 non-null object
Borough                        531 non-null object
Lat                            531 non-null object
Lng                            531 non-null object
Shoe store                     0 non-null float64
Art gallery                    0 non-null float64
College and University         0 non-null float64
Cafe                           0 non-null float64
Vegetarian Vegan Restaurant    0 non-null float64
Travel and Transport           0 non-null float64
dtypes: float64(6), object(4)
memory usage: 41.6+ KB


Now we have prepared everything required to start filling our dataframe with number of venues. Due to Foursquare free account limitations it's not possible to perform all requests straight and at once - we'll exceed the free quota.
One possible way to remain in the limits, without compromising quality of the data, should be to perform the first request with main criteria for all 533 locations and then limit other requests to the top results. As agreed with the client, the main location criteria within the "richest" city is concentration of Shoe stores. 
We'll need to explore data distribution of this criteria and limit next requests only to the top locations.

In [113]:
# let's start with the main criteria number of shoe stores in location
nr_venues('4bf58dd8d48988d107951735','Shoe store')

In [114]:
df_lon3.sort_values(['Shoe store'], ascending=False, inplace=True)
df_lon3.reset_index(drop=True, inplace=True)
df_lon3.head(15)

,Location,Borough,Lat,Lng,Shoe store,Art gallery,College and University,Cafe,Vegetarian Vegan Restaurant,Travel and Transport
0,Fitzrovia,Camden,51.518529819132,-0.13784759302216,50.0,NaN,NaN,NaN,NaN,NaN
1,Soho,Westminster,51.517585379522,-0.13500343088978,50.0,NaN,NaN,NaN,NaN,NaN
2,Mayfair,Westminster,51.508826777815,-0.14977175677031,49.0,NaN,NaN,NaN,NaN,NaN
3,St James's,Westminster,51.508598701761,-0.1353704316591,48.0,NaN,NaN,NaN,NaN,NaN
4,Covent Garden,Westminster,51.512009617972,-0.12370182518566,47.0,NaN,NaN,NaN,NaN,NaN
5,Marylebone (also St Marylebone),Westminster,51.517813528559,-0.1494075869483,47.0,NaN,NaN,NaN,NaN,NaN
6,St Giles,Camden,51.517355462669,-0.1205994650938,40.0,NaN,NaN,NaN,NaN,NaN
7,Holborn,Camden,51.517355462669,-0.1205994650938,40.0,NaN,NaN,NaN,NaN,NaN
8,Aldwych,Westminster,51.512815974418,-0.11790391775668,39.0,NaN,NaN,NaN,NaN,NaN
9,Shepherd's Bush,Hammersmith and Fulham,51.503649663134,-0.22202608436881,33.0,NaN,NaN,NaN,NaN,NaN


There are a lot of locations with no Shoe store available according Foursquare. As agreed with our client, we are not interested in such locations so let's drop them.

In [115]:
# Get names of indexes for which column Shoe store has value 0
indexNames = df_lon3[df_lon3['Shoe store'] == 0 ].index
 
# Delete these row indexes from dataFrame
df_lon3.drop(indexNames , inplace=True)

In [116]:
df_lon3.describe()

,Shoe store,Art gallery,College and University,Cafe,Vegetarian Vegan Restaurant,Travel and Transport
count,144.000000,0.0,0.0,0.0,0.0,0.0
mean,7.104167,NaN,NaN,NaN,NaN,NaN
std,11.998088,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN
25%,1.000000,NaN,NaN,NaN,NaN,NaN
50%,2.000000,NaN,NaN,NaN,NaN,NaN
75%,5.000000,NaN,NaN,NaN,NaN,NaN
max,50.000000,NaN,NaN,NaN,NaN,NaN


Quick Analysis: 50% of Locations in London have between 1 and 5 shoe stores with the average being 7,1. 
Let's drop locations wtih less than 7 Shoe stores.

In [117]:
# Get names of indexes for which column Shoe store has value less than 7
indexNames = df_lon3[df_lon3['Shoe store'] < 7 ].index
 
# Delete these row indexes from dataFrame
df_lon3.drop(indexNames , inplace=True)
df_lon3

,Location,Borough,Lat,Lng,Shoe store,Art gallery,College and University,Cafe,Vegetarian Vegan Restaurant,Travel and Transport
0,Fitzrovia,Camden,51.518529819132,-0.13784759302216,50.0,NaN,NaN,NaN,NaN,NaN
1,Soho,Westminster,51.517585379522,-0.13500343088978,50.0,NaN,NaN,NaN,NaN,NaN
2,Mayfair,Westminster,51.508826777815,-0.14977175677031,49.0,NaN,NaN,NaN,NaN,NaN
3,St James's,Westminster,51.508598701761,-0.1353704316591,48.0,NaN,NaN,NaN,NaN,NaN
4,Covent Garden,Westminster,51.512009617972,-0.12370182518566,47.0,NaN,NaN,NaN,NaN,NaN
5,Marylebone (also St Marylebone),Westminster,51.517813528559,-0.1494075869483,47.0,NaN,NaN,NaN,NaN,NaN
6,St Giles,Camden,51.517355462669,-0.1205994650938,40.0,NaN,NaN,NaN,NaN,NaN
7,Holborn,Camden,51.517355462669,-0.1205994650938,40.0,NaN,NaN,NaN,NaN,NaN
8,Aldwych,Westminster,51.512815974418,-0.11790391775668,39.0,NaN,NaN,NaN,NaN,NaN
9,Shepherd's Bush,Hammersmith and Fulham,51.503649663134,-0.22202608436881,33.0,NaN,NaN,NaN,NaN,NaN


In [118]:
# Now we have only 30 locations left and can fill the rest of the table without getting out of the FourSquare free limit
nr_venues( '4bf58dd8d48988d1e2931735','Art gallery')
nr_venues('4d4b7105d754a06372d81259','College and University')
nr_venues('4bf58dd8d48988d16d941735','Cafe')
nr_venues('4bf58dd8d48988d1d3941735','Vegetarian Vegan Restaurant')
nr_venues('4d4b7105d754a06379d81259','Travel and Transport')
df_lon3

,Location,Borough,Lat,Lng,Shoe store,Art gallery,College and University,Cafe,Vegetarian Vegan Restaurant,Travel and Transport
0,Fitzrovia,Camden,51.518529819132,-0.13784759302216,50.0,49.0,46.0,50.0,18.0,37.0
1,Soho,Westminster,51.517585379522,-0.13500343088978,50.0,48.0,48.0,50.0,22.0,39.0
2,Mayfair,Westminster,51.508826777815,-0.14977175677031,49.0,48.0,23.0,50.0,2.0,43.0
3,St James's,Westminster,51.508598701761,-0.1353704316591,48.0,49.0,33.0,50.0,15.0,40.0
4,Covent Garden,Westminster,51.512009617972,-0.12370182518566,47.0,44.0,45.0,50.0,8.0,48.0
5,Marylebone (also St Marylebone),Westminster,51.517813528559,-0.1494075869483,47.0,48.0,43.0,50.0,6.0,38.0
6,St Giles,Camden,51.517355462669,-0.1205994650938,40.0,38.0,50.0,50.0,10.0,43.0
7,Holborn,Camden,51.517355462669,-0.1205994650938,40.0,38.0,50.0,50.0,10.0,43.0
8,Aldwych,Westminster,51.512815974418,-0.11790391775668,39.0,33.0,50.0,49.0,4.0,37.0
9,Shepherd's Bush,Hammersmith and Fulham,51.503649663134,-0.22202608436881,33.0,0.0,17.0,29.0,3.0,40.0


In [119]:
df_lon3.describe()

,Shoe store,Art gallery,College and University,Cafe,Vegetarian Vegan Restaurant,Travel and Transport
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,25.903226,20.322581,24.354839,38.677419,5.677419,35.709677
std,14.656864,19.156178,17.511041,14.201848,6.784232,10.159375
min,8.000000,0.000000,0.000000,4.000000,0.000000,7.000000
25%,11.000000,1.000000,11.500000,30.500000,1.000000,33.000000
50%,25.000000,16.000000,19.000000,49.000000,2.000000,37.000000
75%,39.500000,38.000000,43.000000,50.000000,9.500000,43.000000
max,50.000000,49.000000,50.000000,50.000000,22.000000,49.000000


Now we can calculate overall rating for the top locations as agreed with the client: Shoe store with weighting coefficient of 0,5 and the rest with 0,1 

In [120]:
df_lon3['Rating'] = df_lon3['Shoe store']*0.5 + (df_lon3['Art gallery']+df_lon3['College and University']+
                                                 df_lon3['Cafe']+df_lon3['Vegetarian Vegan Restaurant']+
                                                 df_lon3['Travel and Transport'] )*0.1
df_lon3.sort_values(['Rating'], ascending=False, inplace=True)

In [121]:
df_lon3

,Location,Borough,Lat,Lng,Shoe store,Art gallery,College and University,Cafe,Vegetarian Vegan Restaurant,Travel and Transport,Rating
1,Soho,Westminster,51.517585379522,-0.13500343088978,50.0,48.0,48.0,50.0,22.0,39.0,45.7
0,Fitzrovia,Camden,51.518529819132,-0.13784759302216,50.0,49.0,46.0,50.0,18.0,37.0,45.0
4,Covent Garden,Westminster,51.512009617972,-0.12370182518566,47.0,44.0,45.0,50.0,8.0,48.0,43.0
3,St James's,Westminster,51.508598701761,-0.1353704316591,48.0,49.0,33.0,50.0,15.0,40.0,42.7
5,Marylebone (also St Marylebone),Westminster,51.517813528559,-0.1494075869483,47.0,48.0,43.0,50.0,6.0,38.0,42.0
2,Mayfair,Westminster,51.508826777815,-0.14977175677031,49.0,48.0,23.0,50.0,2.0,43.0,41.1
6,St Giles,Camden,51.517355462669,-0.1205994650938,40.0,38.0,50.0,50.0,10.0,43.0,39.1
7,Holborn,Camden,51.517355462669,-0.1205994650938,40.0,38.0,50.0,50.0,10.0,43.0,39.1
8,Aldwych,Westminster,51.512815974418,-0.11790391775668,39.0,33.0,50.0,49.0,4.0,37.0,36.8
15,Charing Cross,Westminster,51.508368858456,-0.12096929668046,25.0,44.0,43.0,50.0,3.0,45.0,31.0


In [122]:
df_lon3.dtypes

Location                        object
Borough                         object
Lat                             object
Lng                             object
Shoe store                     float64
Art gallery                    float64
College and University         float64
Cafe                           float64
Vegetarian Vegan Restaurant    float64
Travel and Transport           float64
Rating                         float64
dtype: object

Create a map of London with neighborhoods superimposed on top.

In [123]:
address = 'London, UK'

geolocator = Nominatim(user_agent="lon_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


In [124]:
df_lon3['Lat']=df_lon3['Lat'].astype(float)
df_lon3['Lng']=df_lon3['Lng'].astype(float)

In [125]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=12.5)

# add markers to map
for lat, lng, borough, neighborhood, rating in zip(df_lon3['Lat'], df_lon3['Lng'], df_lon3['Borough'], df_lon3['Location'], df_lon3['Rating']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=rating*0.7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

Results and Discussion

So, the bests locations are Soho, Fitzrovia and Covent Garden
